In [9]:
import pandas as pd
from google.colab import files
import io
uploaded = files.upload()



Saving ad_clicks.csv to ad_clicks (1).csv


In [12]:
#ad_clicks = pd.read_csv('ad_clicks.csv')
ad_clicks = pd.read_csv(io.BytesIO(uploaded['ad_clicks (1).csv'])) #version for Google Colab

Examine the first few rows of ad_clicks.

In [13]:
print(ad_clicks.head())

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google   2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook    7 - Sunday   

  ad_click_timestamp experimental_group  
0               7:18                  A  
1                NaN                  B  
2                NaN                  A  
3                NaN                  B  
4                NaN                  B  



If the column ad_click_timestamp is not null, then someone actually clicked on the ad that was displayed. Createing a new column called is_click, which is True if ad_click_timestamp is not null and False otherwise.

In [16]:
platform_most_clicks = (ad_clicks.groupby('utm_source')\
.user_id.count()\
.reset_index())
print(platform_most_clicks)

  utm_source  user_id
0      email      255
1   facebook      504
2     google      680
3    twitter      215


We want to know the percent of people who clicked on ads from each utm_source.

Starting by grouping by utm_source and is_click and counting the number of user_id‘s in each of those groups. Save your answer to the variable clicks_by_source.

In [17]:
ad_clicks['is_click'] = ~ad_clicks.ad_click_timestamp.isnull()
print(ad_clicks.head())

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google   2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook    7 - Sunday   

  ad_click_timestamp experimental_group  is_click  
0               7:18                  A      True  
1                NaN                  B     False  
2                NaN                  A     False  
3                NaN                  B     False  
4                NaN                  B     False  


Now let’s pivot the data so that the columns are is_click (either True or False), the index is utm_source, and the values are user_id.

Save your results to the variable clicks_pivot.

In [18]:
clicks_by_source = ad_clicks.groupby(['utm_source','is_click']).user_id.count().reset_index()
print(clicks_by_source)

  utm_source  is_click  user_id
0      email     False      175
1      email      True       80
2   facebook     False      324
3   facebook      True      180
4     google     False      441
5     google      True      239
6    twitter     False      149
7    twitter      True       66


Create a new column in clicks_pivot called percent_clicked which is equal to the percent of users who clicked on the ad from each utm_source.

Was there a difference in click rates for each source?

In [19]:
clicks_pivot = pd.pivot_table(clicks_by_source,
                              index = 'utm_source',
                              columns = 'is_click',
                              values = 'user_id').reset_index()
print(clicks_pivot)

is_click utm_source  False  True
0             email    175    80
1          facebook    324   180
2            google    441   239
3           twitter    149    66


The column experimental_group tells us whether the user was shown Ad A or Ad B.

Were approximately the same number of people shown both ads?

In [20]:
clicks_pivot['percent_clicked'] = clicks_pivot[False] / (clicks_pivot[False] + clicks_pivot[True] )
print(clicks_pivot)

is_click utm_source  False  True  percent_clicked
0             email    175    80         0.686275
1          facebook    324   180         0.642857
2            google    441   239         0.648529
3           twitter    149    66         0.693023


Using the column is_click that we defined earlier, check to see if a greater percentage of users clicked on Ad A or Ad B.

In [21]:
user_saw_ab = ad_clicks.groupby(['experimental_group']).user_id.count().reset_index()
print(user_saw_ab)

  experimental_group  user_id
0                  A      827
1                  B      827


The Product Manager for the A/B test thinks that the clicks might have changed by day of the week.

Start by creating two DataFrames: a_clicks and b_clicks, which contain only the results for A group and B group, respectively.

In [22]:
a_clicks = ad_clicks[ad_clicks['experimental_group'] == 'A']
b_clicks = ad_clicks[ad_clicks['experimental_group'] == 'B']

For each group (a_clicks and b_clicks), calculate the percent of users who clicked on the ad by day.

In [23]:
a_clicks_day = a_clicks.groupby(['day','is_click'])\
                        .user_id.count()\
                        .reset_index()\
                        .pivot(index='day',
                              columns='is_click',
                              values='user_id')\
                        .reset_index()
a_clicks_day['percentage'] = a_clicks_day[True] /(a_clicks_day[True] + a_clicks_day[False])
print(a_clicks_day)

b_clicks_day = b_clicks.groupby(['day','is_click'])\
                        .user_id.count()\
                        .reset_index()\
                        .pivot(index='day',
                              columns='is_click',
                              values='user_id')\
                        .reset_index()
b_clicks_day['percentage'] = b_clicks_day[True] /(b_clicks_day[True] + b_clicks_day[False])
print(b_clicks_day)

is_click            day  False  True  percentage
0            1 - Monday     70    43    0.380531
1           2 - Tuesday     76    43    0.361345
2         3 - Wednesday     86    38    0.306452
3          4 - Thursday     69    47    0.405172
4            5 - Friday     77    51    0.398438
5          6 - Saturday     73    45    0.381356
6            7 - Sunday     66    43    0.394495
is_click            day  False  True  percentage
0            1 - Monday     81    32    0.283186
1           2 - Tuesday     74    45    0.378151
2         3 - Wednesday     89    35    0.282258
3          4 - Thursday     87    29    0.250000
4            5 - Friday     90    38    0.296875
5          6 - Saturday     76    42    0.355932
6            7 - Sunday     75    34    0.311927


Compare the results for A and B. What happened over the course of the week?

Do you recommend that your company use Ad A or Ad B?

In [24]:
compare = a_clicks_day.merge(b_clicks_day,
                           left_on = 'day',
                         right_on = 'day',)
compare['difference x/y'] = compare['percentage_x'] / compare['percentage_y']
print(compare)

print(compare['difference x/y'].mean() )
# Ad.B is beter

is_click            day  False_x  True_x  percentage_x  False_y  True_y  \
0            1 - Monday       70      43      0.380531       81      32   
1           2 - Tuesday       76      43      0.361345       74      45   
2         3 - Wednesday       86      38      0.306452       89      35   
3          4 - Thursday       69      47      0.405172       87      29   
4            5 - Friday       77      51      0.398438       90      38   
5          6 - Saturday       73      45      0.381356       76      42   
6            7 - Sunday       66      43      0.394495       75      34   

is_click  percentage_y  difference x/y  
0             0.283186        1.343750  
1             0.378151        0.955556  
2             0.282258        1.085714  
3             0.250000        1.620690  
4             0.296875        1.342105  
5             0.355932        1.071429  
6             0.311927        1.264706  
1.2405641733402375
